In [1]:
!pip install efficient-apriori

In [2]:
# i = 8
# j = 9
# i + j

In [3]:
import pandas as pd
# import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib
import numpy as np

from sklearn.model_selection import GroupKFold
from surprise.model_selection import PredefinedKFold
# from scipy.stats import permutation_test
from sklearn.model_selection import train_test_split
from surprise import Dataset, SVD
from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate
from sklearn import preprocessing
import seaborn as sns
from sklearn.metrics import cohen_kappa_score, mean_absolute_error , mean_squared_error
from efficient_apriori import apriori
import pickle

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
ratings = pd.read_csv("/kaggle/input/bookrecommender/BX-Book-Ratings.csv",encoding='ISO-8859–1',delimiter=";")
# books = pd.read_csv("BX-Books_small.csv",encoding='ISO-8859–1',delimiter=";")
# users = pd.read_csv("/kaggle/input/bookrecommender/BX-Users.csv",encoding='ISO-8859–1',delimiter=";")

In [5]:
len(ratings['User-ID'].unique())

38207

In [6]:
ratings = ratings.rename(columns={"User-ID":"user_id",
                                  "ISBN":"isbn",
                                  "Book-Rating":"book_rating"})
ratings['user_id'] = ratings['user_id'].astype(str)
ratings['isbn'] = ratings['isbn'].astype(str)
ratings['book_rating'] = ratings['book_rating'].astype(int)

ratings_grp = ratings.groupby(by="isbn",as_index=False).count()
ratings_grp = ratings_grp.loc[ratings_grp.user_id > 10, :]
ratings = ratings.loc[ratings.isbn.isin(ratings_grp['isbn']),:]
print(f"******** rating 1 : {len(ratings)}")

ratings_grp2 = ratings.groupby(by="user_id",as_index=False).count()
ratings_grp2 = ratings_grp2.loc[ratings_grp2.isbn > 10, :]
ratings = ratings.loc[ratings.user_id.isin(ratings_grp2['user_id']),:]
print(f"******** rating 1 : {len(ratings)}")

******** rating 1 : 117035
******** rating 1 : 81330


In [7]:
pt = ratings.pivot_table(index="isbn",columns="user_id",values="book_rating").fillna(0)

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
cosin_simi = cosine_similarity(pt)

In [10]:
simi_matrix = pd.DataFrame(cosin_simi,columns=pt.index.tolist(),index=pt.index.tolist())
item_item_matrix = pd.DataFrame(simi_matrix.unstack(),columns=['rating']).reset_index()
item_item_matrix.columns = ["book_id1","book_id2","rating"]